In [403]:
#Loading all the important libraries
import glob
import pympi
from io import open
from conllu import parse_incr, parse
import pandas as pd
import re
import numpy as np
from tqdm import tqdm

In [404]:
#Functions to find different patterns of Reference numbers(annotation numbers since we have to merge some sentences)
def search_annotation_pattern2(sent_id):
    if re.search("\d{1,4}$", sent_id):
        m = re.search("\d{1,4}$", sent_id)
        return(sent_id, "a" + str(m.group()))
    else:
        pass
def search_annotation_pattern1(sent_id):
    if re.search("\d{1,4}\:\d{1,4}$", sent_id):
        m = re.search("\d{1,4}\:\d{1,4}$", sent_id)
        return(sent_id, "a" + str(m.group().split(":")[0]))
    else:
        return(search_annotation_pattern2(sent_id))

def search_annotation_pattern(sent_id):
    if re.search("\d{1,4}\.\d{1,4}$", sent_id):
        m = re.search("\d{1,4}\.\d{1,4}$", sent_id)
#         n = m.group()
#         s1, s2 = n.split(".")
#         s = s1 + ":" + s2
        return(sent_id, "a" + str(m.group().split(".")[0]))
    else:
        return(search_annotation_pattern1(sent_id))

## Matching the sentences on the basis of Annotation Number

In [406]:
eaf_file_list = glob.glob("/Users/ashish/Desktop/Computation Linguistics Project/Pueble-Nahuatl-Manifest/ELAN-files-Final-proofed-and-most-translated/*.*")

In [407]:
from speach import elan
ann_no_with_time = [] 
for file in eaf_file_list:
    file_name = file.split('/')[-1]
    eaf = elan.read_eaf(file)
    for tier in eaf:
        for ann in tier:
            ann_no_with_time.append([ann.ID, ann, ann.from_ts, ann.to_ts, file_name])

In [408]:
sentence_data_eaf_ann = pd.DataFrame(ann_no_with_time, columns = ["ann_no","sentence", "start_time", "end_time", "eaf_file"])

In [409]:
sentence_data_conllu = pd.DataFrame(columns = ["sentence", "sent_id", "ann_no"])
data_list = []
conllu_file = open("/Users/ashish/Desktop/Computation Linguistics Project/apertium-azz/corpora/azz_itml-ud-test.conllu", "r", encoding="utf-8")
for tokenlist in parse_incr(conllu_file):
    data = tokenlist.metadata
    data_list.append(data)
    sentence = tokenlist.metadata['text']
    sent_id = tokenlist.metadata['sent_id']
    sentence_data_conllu = sentence_data_conllu.append({"sentence":sentence, "sent_id":search_annotation_pattern(sent_id)[0],
                                         "ann_no":search_annotation_pattern(sent_id)[1]}, ignore_index = True)


In [411]:
ann_time = []
for c in tqdm(sentence_data_conllu.index):
    for e in sentence_data_eaf_ann.index:
        if (sentence_data_eaf_ann.iloc[e].eaf_file in sentence_data_conllu.iloc[c].sent_id):
            if(sentence_data_eaf_ann.iloc[e].ann_no == sentence_data_conllu.iloc[c].ann_no):
                ann_time.append([sentence_data_conllu.iloc[c].ann_no, sentence_data_eaf_ann.iloc[e].start_time, sentence_data_eaf_ann.iloc[e].end_time, sentence_data_conllu.iloc[c].sentence])

                break

100%|█████████████████████████████████████| 1227/1227 [1:17:54<00:00,  3.81s/it]


In [507]:
ann_time_df = pd.DataFrame(ann_time, columns = ["annotation_no", "start_time", "end_time", "sentence"])

### Removing sentences already matched in above step and performing sentence matching

In [508]:
#accessing conllu test file to read all the sentences we have to match in EAF files
data_list = []
conllu_file = open("/Users/ashish/Desktop/Computation Linguistics Project/apertium-azz/corpora/azz_itml-ud-test.conllu", "r", encoding="utf-8")
for tokenlist in parse_incr(conllu_file):
    data = tokenlist.metadata
    data_list.append(data)
data_df = pd.DataFrame(data_list)

In [509]:
#Selecting important columns and extracting only 
sentence_conllu = data_df.iloc[:,0:5]
ann_no = []
for i in range(len(sentence_conllu)):
    ann = search_annotation_pattern(sentence_conllu.loc[i, "sent_id"])[1]
    ann_no.append(ann)
sentence_conllu["ann_no"] = ann_no

/var/folders/6l/hglj_fjs6n5gn9jr00v55str0000gn/T/ipykernel_67999/2279775885.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentence_conllu["ann_no"] = ann_no


In [510]:
#removing already identified sentences from the conllu list
l = []
for i in sentence_conllu.index:
    for j in ann_time_df.index:
        if(ann_time_df.iloc[j].annotation_no == sentence_conllu.iloc[i].ann_no and ann_time_df.iloc[j].sentence == sentence_conllu.iloc[i].text):
            l.append(i)
sentence_data_c = sentence_conllu.drop(index = l)

In [471]:
#Function to get sent id
def sent(sent_id):
    if len(sent_id.split(":")) == 4:
        sent_id = sent_id.split(":")[0]
    elif len(sent_id.split(":")) == 3:
        sent_id = sent_id.split(":")[0]
    else:
        sent_id = sent_id.split(":")[0]
    
    return sent_id

In [511]:
sentence_data_conllu = sentence_data_c.reset_index(drop = True)
for i in sentence_data_conllu.index:
    sentence_data_conllu.iloc[i]["sent_id"] = sent(sentence_data_conllu.iloc[i]["sent_id"])
sentence_data_conllu.head()

,sent_id,text,text[orig],text[spa],labels,ann_no
0,kiteixmatiltijtok-tamachilis,¿Keniuj Mochijchiuaj ne Sitalimej?,NaN,¿Cómo se forman las estrellas?,complete,a1
1,kiteixmatiltijtok-tamachilis,"Se youal keman takistok uan amo metstona, itec...",NaN,"En una noche despejada y sin luna, desde el ca...",complete-tree check-mark,a3
2,kiteixmatiltijtok-tamachilis,Neje sitalimej no chijchiujtokej kemej ne tona...,NaN,"Estas estrellas son similares a nuestro sol, a...",NaN,a4
3,kiteixmatiltijtok-tamachilis,"Nijin ijpotok uan mixuak mololouaj, nejin teyi...",NaN,Esos ingredientes van formando grumos que van ...,complete-tree,a6
4,kiteixmatiltijtok-tamachilis,"Ijkuak miki, ne sitalin toponi uan seki átomos...",NaN,"Al morir, las estrellas explotan y parte de lo...",NaN,a10


In [512]:
len(sentence_data_conllu)

728

In [423]:
#Function to get spanish tiers
def get_spanish_tiers(eaf):
    #gettting tier names to collect data using tier name
    tiers = eaf.get_tier_names()
    #selecting tier names for spanish sentences
    tiers_spanish = []
    #for loop to obtain all the spanish translations
    #As for one eaf file spanish translation could be done by various persons
    #"Traducción" is the spanish word for translation, so selecting tier on basis of that.
    for i in tiers:
        if("Traducción" in i):
            tiers_spanish.append(i)
    return(tiers_spanish)


In [424]:
#Function to get sentences for all the spanish tier ids
def get_spanish_sentences(eaf):
    #extracting spanish sentences
    tiers_spanish = get_spanish_tiers(eaf)
    
    #empty space holders to collect the data
    data = []

    #For loop to get data for all the IDs in each file
    for i in range(len(tiers_spanish)):
        data = data + eaf.get_annotation_data_for_tier(tiers_spanish[i])
    return (data)

In [474]:
#accessing all the files using eaf and storing all the sentences in same dataframe
all_data = []
for i in range(len(eaf_file_list)):
    file_name = eaf_file_list[i].split('/')[-1]
    eaf = pympi.Elan.Eaf(eaf_file_list[i])
    a = eaf.get_linked_files()
    media_file_link = a[0]["MEDIA_URL"]
    media_file_name = media_file_link.split('/')[-1]
    sentences = get_spanish_sentences(eaf)
    for j in range(len(sentences)):
        l = list(sentences[j])
        l.insert(len(l),file_name)
        l.insert(len(l), media_file_name)
        sentences[j] = tuple(l)
    all_data = all_data + sentences

In [513]:
sentence_data_eaf = pd.DataFrame(all_data, columns = ["start_time", "end_time", "sentence", "org", "eaf_file_name", "audio_file"])

In [514]:
sentence_data_eaf.head()

,start_time,end_time,sentence,org,eaf_file_name,audio_file
0,0,5737,Pues puedes decirme si conoces esta,"Pos kwali tine:chili:s, ne:n, katí:n tiki:xmat...",Tzina_Botan_JVC313-AND308_kowaatapiitsxiwit-Ar...,Tzina_Botan_JVC313-AND308_kowaatapiitsxiwit-Ar...
1,5737,11688,"hierbita este tapits..., la que dices que se u...","xiwtsí:n ne:n [ta]pi:ts..., tikihtowa ika kini...",Tzina_Botan_JVC313-AND308_kowaatapiitsxiwit-Ar...,Tzina_Botan_JVC313-AND308_kowaatapiitsxiwit-Ar...
2,79909,85222,"Vienen, este, bueno, la serpiente sale del lug...","Wi:tseh, ne:n, *bueno** wa:le:wa ne:n kowa:t w...",Tzina_Botan_JVC313-AND308_kowaatapiitsxiwit-Ar...,Tzina_Botan_JVC313-AND308_kowaatapiitsxiwit-Ar...
3,101415,104620,"Y cómo saben, vienen escuchando o","Wa:n ki:ní:n kimatih, ika, wi:tseh kikaktiwi:t...",Tzina_Botan_JVC313-AND308_kowaatapiitsxiwit-Ar...,Tzina_Botan_JVC313-AND308_kowaatapiitsxiwit-Ar...
4,104620,106161,Se buscan.,Mote:mowah.,Tzina_Botan_JVC313-AND308_kowaatapiitsxiwit-Ar...,Tzina_Botan_JVC313-AND308_kowaatapiitsxiwit-Ar...


In [515]:
from copy import deepcopy
sdee = deepcopy(sentence_data_eaf)
sdcc = deepcopy(sentence_data_conllu)

sentence = set()
from collections import defaultdict
is_sde_used = defaultdict(lambda: False)
is_sdc_used = defaultdict(lambda: False)

for sde in sdee.itertuples():
    is_sde_used[(sde)] = False
    
for sdc in sdcc.itertuples():
    is_sdc_used[(sdc)] = False


The below code is to compare the sent_id and text of eaf file with conllu files(in the first for loop).
In the second for loop we have same snetences repeated many times and we cannot distinguish them with sent_id, so I am just assigning the time I am getting for the first search value.

In [ ]:

for sdc in sdcc.itertuples():
    if not is_sdc_used[(sdc)]:
        for sde in sdee.itertuples():
            if not is_sde_used[(sde)]:
                if sdc[4] == sde[3]:
                    if sde[5] in sdc[1]:
                        sentence.add((sdc[6], sde[1], sde[2], sde[3]))
                        is_sde_used[(sde)] = True
                        is_sdc_used[(sdc)] = True
                        break


for sdc in sdcc.itertuples():
    if not is_sdc_used[(sdc)]:
        for sde in sdee.itertuples():
            if not is_sde_used[(sde)]:
                if sdc[4] == sde[3]:
                    sentence.add((sdc[6], sde[1], sde[2], sde[3]))
                    is_sde_used[(sde)] = True
                    is_sdc_used[(sdc)] = True
                    break

In [517]:
# count_sde = 0 
# for i in is_sde_used.values():
#     if i == True:
#         count_sde += 1
        
# count_sdc = 0 
# for i in is_sdc_used.values():
#     if i == True:
#         count_sdc += 1
# print(count_sde, count_sdc)

5 5


In [519]:
sent_time_df = pd.DataFrame(sentence, columns = ["annotation_no", "start_time", "end_time", "sentence"])

In [530]:
sentence_data_conllu = sentence_data_c.reset_index(drop = True)
for i in sentence_data_conllu.index:
    sentence_data_conllu.iloc[i]["sent_id"] = sent(sentence_data_conllu.iloc[i]["sent_id"])
sentence_data_conllu.head()

,sent_id,text,text[orig],text[spa],labels,ann_no
0,kiteixmatiltijtok-tamachilis,¿Keniuj Mochijchiuaj ne Sitalimej?,NaN,¿Cómo se forman las estrellas?,complete,a1
1,kiteixmatiltijtok-tamachilis,"Se youal keman takistok uan amo metstona, itec...",NaN,"En una noche despejada y sin luna, desde el ca...",complete-tree check-mark,a3
2,kiteixmatiltijtok-tamachilis,Neje sitalimej no chijchiujtokej kemej ne tona...,NaN,"Estas estrellas son similares a nuestro sol, a...",NaN,a4
3,kiteixmatiltijtok-tamachilis,"Nijin ijpotok uan mixuak mololouaj, nejin teyi...",NaN,Esos ingredientes van formando grumos que van ...,complete-tree,a6
4,kiteixmatiltijtok-tamachilis,"Ijkuak miki, ne sitalin toponi uan seki átomos...",NaN,"Al morir, las estrellas explotan y parte de lo...",NaN,a10


In [532]:
len(sentence_data_conllu)

728

In [533]:
def merge_sent(data):
    dicto = defaultdict(lambda: [])
    for _ in data:
        dicto[_[1]].append(_[0])
    data = []
    for key, val in dicto.items():
        data.append((' '.join(val), key))
    return data

In [549]:
df_sent_id = sentence_data_conllu.groupby("sent_id")

In [535]:
frames=[]
for id in sentence_data_conllu.sent_id.unique():
    df_new = df_sent_id.get_group(id)
    df_new1 = df_new[["text", "ann_no"]]
    data = df_new1.values.tolist()
    for i in range(len(data)):
        data[i][0] = str(data[i][0])
    data = merge_sent(data)
    data_df = pd.DataFrame(data, columns = ["sentence", "ann_no"])
    df = df_new.merge(right = data_df, on = "ann_no")
    frames = frames + df.values.tolist()

In [536]:
sentence_data_conllu = pd.DataFrame(frames, columns = ['sent_id', 'text', 'text[org]', 'text[spa]', 'labels', 'ann_no', 'sentence'])

In [537]:
from copy import deepcopy
sdee = deepcopy(sentence_data_eaf)
sdcc = deepcopy(sentence_data_conllu)

sentence = set()
from collections import defaultdict
is_sde_used = defaultdict(lambda: False)
is_sdc_used = defaultdict(lambda: False)

for sde in sdee.itertuples():
    is_sde_used[(sde)] = False
    
for sdc in sdcc.itertuples():
    is_sdc_used[(sdc)] = False


As we did for the above step we are doing same here.

In [548]:
for sdc in sdcc.itertuples():
    if not is_sdc_used[(sdc)]:
        for sde in sdee.itertuples():
            if not is_sde_used[(sde)]:
                if sdc[7] == sde[3]:
                    if sde[5] in sdc[1]:
                        sentence.add((sdc[6], sde[1], sde[2], sde[3]))
                        is_sde_used[(sde)] = True
                        is_sdc_used[(sdc)] = True
                        break


for sdc in sdcc.itertuples():
    if not is_sdc_used[(sdc)]:
        for sde in sdee.itertuples():
            if not is_sde_used[(sde)]:
                if sdc[7] == sde[3]:
                    sentence.add((sdc[6], sde[1], sde[2], sde[3]))
                    is_sde_used[(sde)] = True
                    is_sdc_used[(sdc)] = True
                    break

In [550]:
count_sde = 0 
for i in is_sde_used.values():
    if i == True:
        count_sde += 1
        
count_sdc = 0 
for i in is_sdc_used.values():
    if i == True:
        count_sdc += 1
print(count_sde, count_sdc)

354 354


In [541]:
sent_merge_df = pd.DataFrame(sentence, columns = ["annotation_no", "start_time", "end_time", "sentence"])

1227

In [545]:
frames = [sent_merge_df, sent_time_df, ann_time_df]

In [546]:
final_data = pd.concat(frames)

In [551]:
### set_meta method of pyconll to write back to conllu files with time.

In [552]:
import pyconll

In [554]:
conll_file_path = "/Users/ashish/Desktop/Computation Linguistics Project/apertium-azz/corpora/test.conllu"

In [555]:
test = pyconll.load_from_file(conll_file_path)

The below for loops are assigning start time and end time to sentences we have accumulated in the above steps to the test conllu file.

In [567]:
#Setting start time and end time for the texts
for i in final_data.index:
    for sentence in test:
        if sentence.meta_present("labels"):
            ann_no = sentence.meta_value("labels").split(" ")[0]
            sent = sentence.meta_value("text")
            if (sent == final_data.iloc[i].sentence and ann_no == final_data.iloc[i].annotation_no):
                sentence.set_meta("start_time", final_data.iloc[i].start_time)
                sentence.set_meta("end_time", final_data.iloc[i].start_time)

In [566]:
#setting start time and end time for the text in spanish
for i in final_data.index:
    for sentnece in test:
        if not sentence.meta_present("start_time"):
            if sentence.meta_present("text[spa]"):
                if (sent == final_data.iloc[i].sentence):
                    sentence.set_meta("start_time", final_data.iloc[i].start_time)
                    sentence.set_meta("end_time", final_data.iloc[i].start_time)